# Word2vec+CNN做文本分类

论文详见《Convolutional Neural Networks for Sentence Classification》
http://arxiv.org/abs/1408.5882

Theano完成的代码版本：
https://github.com/yoonkim/CNN_sentence

TensorFlow改写的代码版本：
https://github.com/dennybritz/cnn-text-classification-tf

添加分词和中文词向量映射之后，可用于中文文本分类(情感分析)

#### 本节内容主要记录TensorFlow的版本

### TensorFlow版本介绍：
本节的代码实现主要基于“<a href='http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/'>在TensorFlow中实现CNN进行文本分类</a>”的论文。

具体的思想和内容，参见论文，此处会酌情提取相关内容。

我们将实现一个类似于Kim Yoon的<a href='https://arxiv.org/abs/1408.5882'>用于句子分类</a>的<a href='https://arxiv.org/abs/1408.5882'>卷积神经网络</a>的模型。

本文提出的模型在一系列文本分类任务（如情感分析）中实现了良好的分类性能，并已成为新文本分类体系结构的标准基线。

我假设您已经熟悉应用于NLP的卷积神经网络的基础知识。如果没有，我建议首先阅读了解<a href='http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/'>NLP的卷积神经网络</a>，以获得必要的背景知识。

### Step 1: 数据和预处理 Data and Preprocessing

我们将在这篇文章中使用的数据集是来自<a href='http://www.cs.cornell.edu/people/pabo/movie-review-data/'>烂番茄的电影评论数据</a>- 原始论文中也使用的数据集之一。

数据集包含10,662个示例评论句子，半正面和半负面。数据集的大小约为20k。请注意，由于此数据集非常小，我们可能会过度使用强大的模型。此外，数据集没有官方训练集/测试集拆分，因此我们只使用10％的数据作为开发集dev set。原始论文显示了对数据进行10倍交叉验证(10-fold cross-validation)的结果。



我不会在这篇文章中讨论数据预处理代码，但它可以在Github上获得并执行以下操作：

1. 从原始数据文件中加载正面和负面的句子。
2. 使用与原始论文<a href='https://github.com/yoonkim/CNN_sentence'>Github: 相同的代码</a>清理文本数据。
3. 将每个句子填充到最大句子长度59。我们将特殊<PAD\>标记附加到所有其他句子，使它们成为59个单词的句子。将句子填充到相同长度是有用的，因为它允许我们有效地批量处理我们的数据，因为批处理中的每个示例必须具有相同的长度。
4. 构建词汇索引并将每个单词映射到0到18,765之间的整数（词汇量大小）。每个句子都成为整数向量Each sentence becomes a vector of integers.

In [42]:
import numpy as np
import pickle
from collections import defaultdict
import sys, re
import pandas as pd

In [43]:
def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip() if TREC else string.strip().lower()

In [44]:
def clean_str_sst(string):
    """
    Tokenization/string cleaning for the SST dataset
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)   
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [45]:
def build_data_cv(data_folder, cv=10, clean_string=True):
    '''
    Loads data and split into 10 folds.
    '''
    revs = []
    pos_file = data_folder[0]
    neg_file = data_folder[1]
    
    vocab = defaultdict(float)
    
    with open(pos_file, 'r') as f:
        for line in f:
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(' '.join(rev))
            else:
                orig_rev = ' '.join(rev).lower()
            
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
                
            datum = {'y':1, 'text':orig_rev, 'num_words':len(orig_rev.split()), 'split':np.random.randint(0, cv)}
            revs.append(datum)
    
    with open(neg_file, 'r') as f:
        for line in f:
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(' '.join(rev))
            else:
                orig_rev = ' '.join(rev).lower()
            
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum = {'y':0, 'text': orig_rev, 'num_words':len(orig_rev.split()), 'split':np.random.randint(0,cv)}
            revs.append(datum)
    return revs, vocab

In [46]:
def get_W(word_vecs, k=300):
    '''
    Get word matrix. W[i] is the vector for word indexed by i.
    '''
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size + 1, k), dtype='float32')
    W[0] = np.zeros(k, dtype='float32')
    
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

In [47]:
def load_bin_vec(fname, vocab):
    '''
    Loads 300*1 word vecs from Google (Mikolov) word2vec
    '''
    word_vecs = {}
    
    with open(fname, 'rb') as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
                
    return word_vecs
    

In [48]:
def add_unknown_words(word_vecs, vocab, min_df =1, k=300):
    '''
    For words that occur in at least min_df documents, create a separate word vector. 
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones.
    '''
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
        

第一步：加载正面和负面评论，清理文本，将其添加到一个字典中，并通过np.random.randint(0,10)生成的随机值来为将来的k-fold交叉验证数据抽取，并详细记录其信息。

In [49]:
data_folder = ["./rt-polaritydata/rt-polarity.pos","./rt-polaritydata/rt-polarity.neg"]    
print("loading data...")

revs, vocab = build_data_cv(data_folder, cv=10, clean_string=True)

loading data...


In [53]:
revs[-1]

{'y': 0,
 'text': "enigma is well made , but it 's just too dry and too placid",
 'num_words': 14,
 'split': 5}

In [55]:
print("data loaded!")

data loaded!


第二步：记录数据集基本特征信息：词汇表大小，语料的句子数量，最常句子长度

In [58]:
pd.DataFrame(revs).head()

,num_words,split,text,y
0,34,9,the rock is destined to be the 21st century 's...,1
1,38,4,the gorgeously elaborate continuation of the l...,1
2,5,5,effective but too tepid biopic,1
3,20,8,if you sometimes like to go to the movies to h...,1
4,22,3,"emerges as something rare , an issue movie tha...",1


In [59]:
max_l = np.max(pd.DataFrame(revs)["num_words"])

print("number of sentences: " + str(len(revs)))
print("vocab size: " + str(len(vocab)))
print("max sentence length: " + str(max_l))


number of sentences: 10662
vocab size: 18764
max sentence length: 56


加载现成的已经预训练好的word2vec模型，并将词汇在word2vec模型中找到对应的词汇向量，构造我们数据集的word2vec矩阵。

In [ ]:
w2v_file = sys.argv[1]  # 1.5 G
print("loading word2vec vectors...")

w2v = load_bin_vec(w2v_file, vocab)
print("word2vec loaded!")
print("num words already in word2vec: " + str(len(w2v)))
    

In [ ]:
add_unknown_words(w2v, vocab)
W, word_idx_map = get_W(w2v)
rand_vecs = {}
add_unknown_words(rand_vecs, vocab)
W2, _ = get_W(rand_vecs)
cPickle.dump([revs, W, W2, word_idx_map, vocab], open("mr.p", "wb"))
print("dataset created!")

### Step 2: 模型 The Model

我们将在这篇文章中构建的网络大致如下：

<img src='./images/sclf01.png' width='90%'/>

The first layers embeds words into low-dimensional vectors.   
第一层将单词嵌入到低维向量中。 

The next layer performs convolutions over the embedded word vectors using multiple filter sizes. For example, sliding over 3, 4 or 5 words at a time.  
下一层使用多个滤波器大小对嵌入的字向量执行卷积。例如，一次滑动3个，4个或5个字。

Next, we max-pool the result of the convolutional layer into a long feature vector, add dropout regularization, and classify the result using a softmax layer.   
接下来，我们将卷积层的结果最大化为长特征向量，添加丢失正则化，并使用softmax层对结果进行分类。


因为这是一篇教育性的帖子，所以我决定从原始论文中简化模型：

- 我们不会使用预先训练过的word2vec向量进行单词嵌入。相反，我们从头开始学习嵌入。
- 我们不会对权重向量强制执行L2范数约束。对句子分类的卷积神经网络（和从业者指南）的敏感性分析发现，约束对最终结果影响不大。
- 原始论文用两个输入数据通道进行实验 - 静态和非静态字向量。我们只使用一个channel。

在这里向代码添加上述扩展是相对简单的（几十行代码）。

---

#### Implementation:

为了允许各种超参数配置，我们将代码放入一个TextCNN类中，在init函数中生成模型图。

In [60]:
import tensorflow as tf
import numpy as np

/Users/zoe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
class TextCNN(object):
    '''
    A CNN for text classification. 
    Uses an embedding layers, followed by a convolutional, max-pooling and softmax layer.
    '''
    def __int__(self, sequence_length, num_classes, vocab_size, embedding_size, filter_sizes, num_filters):
        # Implementation...
        # Placeholders for input, output and dropout
        
        # usage: placeholder(dtype, shape=None, name=None)
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        self.dropuout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        

为了实例化该类，我们传递以下参数：

- sequence_length – 我们句子的长度。请记住，我们将所有句子填充为相同的长度（我们的数据集为59）。
- num_classes – 输出层中的类数，在我们的例子中为两个（positive和negative）。
- vocab_size – 我们词汇量的大小。这需要定义嵌入层 embedding layer的大小，嵌入层将具有形状[vocabulary_size, embedding_size]。
- embedding_size – 嵌入的维度。
- filter_sizes – 我们希望卷积filters覆盖的单词数。我们将为num_filters此处指定的每个尺寸。例如，[3, 4, 5]意味着我们将分别对3个，4个和5个单词过滤，总共进行3 * num_filters过滤
- num_filters – 每个过滤器大小的过滤器数量。

tf.placeholder创建一个占位符变量，当我们在train或test时执行它时，我们将其提供给网络。

第二个参数是输入张量的形状。
**None意味着该维度的长度可以是任何东西。**      
在我们的例子中，第一个维度是batch size，并且使用None允许网络处理任意大小的批次。

The probability of keeping a neuron in the dropout layer is also an input to the network because we enable dropout only during training.   
在dropout layer中保留神经元的概率也是网络的输入，因为我们仅在训练期间启用丢失。我们在评估模型时禁用它（稍后会详细介绍）。